# Receptive field size of CNN

In [25]:
#[kernel, stride, padding]

CNN2 = {
    'conv1': [3,1,1],
    'pool1': [3,2,1],
    'conv2': [3,1,1],
    'pool2': [3,2,1],
    'fc': [1,1,0],
}

import math;

class ReceptiveFieldCalculator():
    def calculate(self, architecture, input_image_size):
        input_layer = ('input_layer', input_image_size, 1, 1, 0.5)
        self._print_layer_info(input_layer)
        
        for key in architecture:
            current_layer = self._calculate_layer_info(architecture[key], input_layer, key)
            self._print_layer_info(current_layer)
            input_layer = current_layer
            
    def _print_layer_info(self, layer):
        print(f'------')
        print(f'{layer[0]}: n = {layer[1]}; r = {layer[2]}; j = {layer[3]}; start = {layer[4]}')     
        print(f'------')
            
    def _calculate_layer_info(self, current_layer, input_layer, layer_name):
        n_in = input_layer[1]
        j_in = input_layer[2]
        r_in = input_layer[3]
        start_in = input_layer[4]
        
        k = current_layer[0]
        s = current_layer[1]
        p = current_layer[2]

        n_out = math.floor((n_in - k + 2*p)/s) + 1
        padding = (n_out-1)*s - n_in + k 
        p_right = math.ceil(padding/2)
        p_left = math.floor(padding/2)

        j_out = j_in * s
        r_out = r_in + (k - 1)*j_in
        start_out = start_in + ((k-1)/2 - p_left)*j_in
        return layer_name, n_out, j_out, r_out, start_out

In [27]:
calculator = ReceptiveFieldCalculator()
calculator.calculate(CNN2, 1000)

------
input_layer: n = 1000; r = 1; j = 1; start = 0.5
------
------
conv1: n = 1000; r = 1; j = 3; start = 0.5
------
------
pool1: n = 500; r = 2; j = 5; start = 1.5
------
------
conv2: n = 500; r = 2; j = 9; start = 1.5
------
------
pool2: n = 250; r = 4; j = 13; start = 3.5
------
------
fc: n = 250; r = 4; j = 13; start = 3.5
------


In [28]:
calculator.calculate(CNN2, 6)

------
input_layer: n = 6; r = 1; j = 1; start = 0.5
------
------
conv1: n = 6; r = 1; j = 3; start = 0.5
------
------
pool1: n = 3; r = 2; j = 5; start = 1.5
------
------
conv2: n = 3; r = 2; j = 9; start = 1.5
------
------
pool2: n = 2; r = 4; j = 13; start = 1.5
------
------
fc: n = 2; r = 4; j = 13; start = 1.5
------


### for CNN3 (cpcplll), receptive field size of 4, 4
### for CNN2 (cplll), receptive field size of 2, 2

## Another way to calculate receptive field size

In [7]:
# [filter size, stride, padding]
#Assume the two dimensions are the same
#Each kernel requires the following parameters:
# - k_i: kernel size
# - s_i: stride
# - p_i: padding (if padding is uneven, right padding will higher than left padding; "SAME" option in tensorflow)
# 
#Each layer i requires the following parameters to be fully represented: 
# - n_i: number of feature (data layer has n_1 = imagesize )
# - j_i: distance (projected to image pixel distance) between center of two adjacent features
# - r_i: receptive field of a feature in layer i
# - start_i: position of the first feature's receptive field in layer i (idx start from 0, negative means the center fall into padding)

import math
convnet =   [[3,2,1],[3,1,2],[1, 1, 0]]
layer_names = ['conv1','pool1', 'fc']
imsize = 100

def outFromIn(conv, layerIn):
  n_in = layerIn[0]
  j_in = layerIn[1]
  r_in = layerIn[2]
  start_in = layerIn[3]
  k = conv[0]
  s = conv[1]
  p = conv[2]
  
  n_out = math.floor((n_in - k + 2*p)/s) + 1
  actualP = (n_out-1)*s - n_in + k 
  pR = math.ceil(actualP/2)
  pL = math.floor(actualP/2)
  
  j_out = j_in * s
  r_out = r_in + (k - 1)*j_in
  start_out = start_in + ((k-1)/2 - pL)*j_in
  return n_out, j_out, r_out, start_out
  
def printLayer(layer, layer_name):
  print(layer_name + ":")
  print("\t n features: %s \n \t jump: %s \n \t receptive size: %s \t start: %s " % (layer[0], layer[1], layer[2], layer[3]))
 
layerInfos = []
if __name__ == '__main__':
#first layer is the data layer (image) with n_0 = image size; j_0 = 1; r_0 = 1; and start_0 = 0.5
  print ("-------Net summary------")
  currentLayer = [imsize, 1, 1, 0.5]
  printLayer(currentLayer, "input image")
  for i in range(len(convnet)):
    currentLayer = outFromIn(convnet[i], currentLayer)
    layerInfos.append(currentLayer)
    printLayer(currentLayer, layer_names[i])
  print ("------------------------")
  layer_name = input ("Layer name where the feature in: ")
  layer_idx = layer_names.index(layer_name)
  idx_x = int(input ("index of the feature in x dimension (from 0)"))
  idx_y = int(input ("index of the feature in y dimension (from 0)"))
  
  n = layerInfos[layer_idx][0]
  j = layerInfos[layer_idx][1]
  r = layerInfos[layer_idx][2]
  start = layerInfos[layer_idx][3]
  assert(idx_x < n)
  assert(idx_y < n)
  
  print ("receptive field: (%s, %s)" % (r, r))
  print ("center: (%s, %s)" % (start+idx_x*j, start+idx_y*j))
  

-------Net summary------
input image:
	 n features: 100 
 	 jump: 1 
 	 receptive size: 1 	 start: 0.5 
conv1:
	 n features: 50 
 	 jump: 2 
 	 receptive size: 3 	 start: 1.5 
pool1:
	 n features: 52 
 	 jump: 2 
 	 receptive size: 7 	 start: -0.5 
fc:
	 n features: 52 
 	 jump: 2 
 	 receptive size: 7 	 start: -0.5 
------------------------
receptive field: (3, 3)
center: (3.5, 1.5)
